In [1]:
import pandas as pd
import numpy as np
import json
import sys
import random

import ast

import pymysql

from datetime import date

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import matplotlib.patheffects as path_effects
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import to_rgba
from matplotlib import cm
from matplotlib import colorbar
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.patches import RegularPolygon
from matplotlib.patches import ArrowStyle
from matplotlib.patches import Circle

from matplotlib.colors import Normalize
import matplotlib.patheffects as pe

import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from mplsoccer import VerticalPitch, Pitch, Radar, FontManager, grid, PyPizza

import scipy.stats as stats

from highlight_text import  ax_text, fig_text

from soccerplots.utils import add_image

from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing

import abs_import

import statistics
import math
import ipywidgets as widgets

from IPython.display import display, Math, Latex

import warnings

plt.rcParams["figure.dpi"] = 300

from matplotlib import font_manager

from app import *

sys.path.append('Functions/visualization')

from Functions.utils import read_json
from Functions.visualization.passing_network import draw_pitch, draw_pass_map

clubColors = {'Brazil' : ['#fadb04', '#1c3474'],
              'Portugal' : ['#e1231b', '#004595'],
              'Argentina' : ['#52a9dc', '#dbe4ea'],
              'Saudi Arabia' : ['#145735', '#dbe4ea'],
              'Ghana' : ['#145735', '#dbe4ea'],
              'Serbia' : ['#FF0000', '#E8E8E8'],
              'Spain' : ['#FF0000', '#E8E8E8'],
              'Germany' : ['#aa9e56', '#FF0000'],
              'France' : ['#202960', '#d10827'],
              'Poland' : ['#d10827', '#E8E8E8'],
              'Morocco' : ['#db221b', '#044c34'],
              'Croatia' : ['#e71c23', '#3f85c5'],
              'Netherlands' : ['#f46c24', '#dcd9d7'],
              'Senegal' : ['#34964a', '#eedf36'],
              'Denmark' : ['#cb1617', '#E8E8E8'],
              'Iran' : ['#269b44', '#dd1212'],
              'Belgium' : ['#ff0000', '#e30613'],
              'USA' : ['#ff0000', '#202960'],
              'Switzerland' : ['#ff0000', '#e8e8e8'],
              'Australia' : ['#202960', '#e30613'],
              'Wales' : ['#ff0000', '#e8e8e8'],
              'Mexico' : ['#00a94f', '#ff0000'],
              'Uruguay' : ['#52a9dc', '#e8e8e8'],
              'Canada' : ['#ff0000', '#ff0000'],
              'Costa Rica' : ['#ff0000', '#202960']}

df1 = pd.read_csv('Data/Home/Mundial/homeGameformations.csv')
df1.rename({'playerIds' : 'playerId'}, axis=1, inplace=True)
df2 = pd.read_csv('Database/optaMundial.csv')
df = pd.concat([df1, df2], axis=0)

In [3]:
def pad_list(lst, max_len):
    return lst + [lst[-1]] * (max_len - len(lst))

df3 = df1[['formationId', 'jerseyNumbers', 'formationName', 'playerId', 'formationSlots', 'startMinuteExpanded', 'endMinuteExpanded']]

df3[['startMinuteExpanded', 'endMinuteExpanded', 'formationId']] = df3[['startMinuteExpanded', 'endMinuteExpanded', 'formationId']].astype(str)

new_df = df3[['formationSlots', 'startMinuteExpanded', 'endMinuteExpanded', 'formationId', 'jerseyNumbers', 'playerId']].apply(lambda row: row.apply(lambda item: pad_list(item.split(','), max(row.str.contains(',') + 1))), axis=1).explode(df3[['formationSlots', 'jerseyNumbers', 'playerId']].columns.tolist())

new_df['formationSlots'] = new_df['formationSlots'].str.replace("[[]","")
new_df['formationSlots'] = new_df['formationSlots'].str.replace("[]]","")

new_df['jerseyNumbers'] = new_df['jerseyNumbers'].str.replace("[[]","")
new_df['jerseyNumbers'] = new_df['jerseyNumbers'].str.replace("[]]","")

new_df['playerId'] = new_df['playerId'].str.replace("[[]","")
new_df['playerId'] = new_df['playerId'].str.replace("[]]","")

new_df['formationSlots'] = new_df['formationSlots'].astype(int)
new_df['jerseyNumbers'] = new_df['jerseyNumbers'].astype(int)
new_df['playerId'] = new_df['playerId'].astype(int)

new_df = new_df.loc[new_df.formationSlots != 0].reset_index(drop=True)

In [59]:
new_df['formationName'] = 'tactic'
for idx, row in enumerate(new_df.formationId):
    if row == ['4', '4']:
        new_df['formationName'][idx] = '433'

In [61]:
new_df['position'] = 'Pos'
for idx, row in enumerate(new_df.formationSlots):
    if (row == 1) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'RB'
    elif (row == 3) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'LB'
    elif (row == 4) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'DMC'
    elif (row == 5) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'RCB'
    elif (row == 6) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'LCB'
    elif (row == 7) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'RCM'
    elif (row == 8) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'LCM'
    elif (row == 9) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'CF'
    elif (row == 10) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'RW'
    elif (row == 11) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'LW'

In [ ]:
df1[['formationId', 'formationName', 'playerId', 'formationSlots', 'startMinuteExpanded', 'endMinuteExpanded']]